In [0]:
from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip
from pyspark.sql.functions import lit

# Khởi tạo SparkSession với tất cả cấu hình cần thiết
builder = SparkSession.builder \
    .appName("WeatherPrediction") \
    .master("local[*]") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.driver.memory", "12g") \
    .config("spark.executor.memory", "12g")

# Áp dụng cấu hình Delta và khởi tạo SparkSession
spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [0]:
spark.sparkContext.setLogLevel("ERROR")

In [0]:
# Đường dẫn đến HDFS trên localhost:9000
delta_table_path = "dbfs:/minhhieu/delta/gold/weather_features"


# Đọc lại dữ liệu từ Delta Table
df = spark.read.format("delta").load(delta_table_path)

from pyspark.sql.functions import col


num_rows = df.count()

df = df.limit(num_rows - 1)

df.show()

+-----+-----+----+-----------+---------+----+---------+----+-----+--------+-------------+--------+--------------------+----+------------------------+----------------------+-----------------+----------------------+-----------------+------------------+---------------------+--------------------------+---------------------+--------------------+--------------------+-----------------+
| time|month|year|temperature|feelslike|wind|direction|gust|cloud|humidity|precipitation|pressure|             weather|Rain|temperature_after_3_hour|feelslike_after_3_hour|wind_after_3_hour|direction_after_3_hour|gust_after_3_hour|cloud_after_3_hour|humidity_after_3_hour|precipitation_after_3_hour|pressure_after_3_hour|weather_after_3_hour|               label|rain_after_3_hour|
+-----+-----+----+-----------+---------+----+---------+----+-----+--------+-------------+--------+--------------------+----+------------------------+----------------------+-----------------+----------------------+-----------------+-----

In [0]:
# Đếm số lượng cột
num_columns = len(df.columns)
print(f"Số lượng cột: {num_columns}")

# Đếm số lượng dòng
num_rows = df.count()
print(f"Số lượng dòng: {num_rows}")


Số lượng cột: 26
Số lượng dòng: 8510


In [0]:
from pyspark.ml.feature import StringIndexer

# Tạo bộ chuyển đổi cho các cột
indexers = [
    StringIndexer(inputCol="direction", outputCol="direction_index"),
    StringIndexer(inputCol="weather", outputCol="weather_index"),
    StringIndexer(inputCol="direction_after_3_hour", outputCol="direction_after_index"),
    StringIndexer(inputCol="weather_after_3_hour", outputCol="weather_after_index"),
]

# Áp dụng pipeline để chuyển đổi tất cả các cột cùng lúc
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=indexers)
df = pipeline.fit(df).transform(df)

# Trích xuất ánh xạ thành dictionary
direction_mapping = df.select("direction", "direction_index").distinct().collect()
weather_mapping = df.select("weather", "weather_index").distinct().collect()
direction_after_mapping = df.select("direction_after_3_hour", "direction_after_index").distinct().collect()
weather_after_mapping = df.select("weather_after_3_hour", "weather_after_index").distinct().collect()

# Lưu vào dictionary
direction_dict = {row["direction"]: int(row["direction_index"]) for row in direction_mapping}
weather_dict = {row["weather"]: int(row["weather_index"]) for row in weather_mapping}
direction_after_dict = {row["direction_after_3_hour"]: int(row["direction_after_index"]) for row in direction_after_mapping}
weather_after_dict = {row["weather_after_3_hour"]: int(row["weather_after_index"]) for row in weather_after_mapping}

# In kết quả
print("Mapping Direction:", direction_dict)
print("Mapping Weather:", weather_dict)
print("Mapping Direction After 3 Hours:", direction_after_dict)
print("Mapping Weather After 3 Hours:", weather_after_dict)


Mapping Direction: {'ENE': 8, 'NE': 10, 'NNE': 9, 'ESE': 3, 'E': 5, 'SE': 0, 'SSE': 4, 'NNW': 14, 'WSW': 1, 'S': 11, 'WNW': 12, 'NW': 15, 'W': 7, 'SW': 2, 'SSW': 6, 'N': 13}
Mapping Weather: {'Clear': 1, 'Sunny': 2, 'Partly cloudy': 0, 'Cloudy': 5, 'Patchy rain possible': 3, 'Light rain shower': 6, 'Overcast': 7, 'Moderate or heavy rain shower': 4, 'Patchy light drizzle': 12, 'Torrential rain shower': 9, 'Light drizzle': 14, 'Patchy light rain': 10, 'Thundery outbreaks possible': 8, 'Light rain': 13, 'Patchy light rain with thunder': 11, 'Moderate rain': 17, 'Moderate rain at times': 16, 'Mist': 15, 'Heavy rain at times': 19, 'Heavy rain': 18}
Mapping Direction After 3 Hours: {'NE': 10, 'NNE': 9, 'ENE': 8, 'ESE': 3, 'E': 5, 'SE': 0, 'SSE': 4, 'NNW': 14, 'WSW': 1, 'S': 11, 'WNW': 12, 'NW': 15, 'W': 7, 'SW': 2, 'SSW': 6, 'N': 13}
Mapping Weather After 3 Hours: {'Clear': 1, 'Sunny': 2, 'Partly cloudy': 0, 'Cloudy': 5, 'Patchy rain possible': 3, 'Light rain shower': 6, 'Overcast': 7, 'Mode

In [0]:
# Thay thế cột chuỗi bằng cột đã mã hóa
df = df \
    .drop("direction", "weather", "direction_after_3_hour", "weather_after_3_hour") \
    .withColumnRenamed("direction_index", "direction") \
    .withColumnRenamed("weather_index", "weather") \
    .withColumnRenamed("direction_after_index", "direction_after_3_hour") \
    .withColumnRenamed("weather_after_index", "weather_after_3_hour")


In [0]:
from pyspark.sql.functions import regexp_replace

# Sửa cột 'time', loại bỏ ':00'
df = df.withColumn("time", regexp_replace("time", ":00", ""))


In [0]:
from pyspark.sql.functions import col
from pyspark.sql import DataFrame

def cast_columns_to_float(df: DataFrame, columns: list) -> DataFrame:
    for c in columns:
        df = df.withColumn(c, col(c).cast("float"))
    return df


In [0]:
from pyspark.sql.functions import sin, cos, col
import math

# Chuyển đổi cột 'time' (giờ trong ngày) thành sin/cos
df = df.withColumn("time_sin", sin(2 * math.pi * col("time") / 24))
df = df.withColumn("time_cos", cos(2 * math.pi * col("time") / 24))

# Chuyển đổi cột 'month' (tháng trong năm) thành sin/cos
df = df.withColumn("month_sin", sin(2 * math.pi * col("month") / 12))
df = df.withColumn("month_cos", cos(2 * math.pi * col("month") / 12))


In [0]:
input_cols = [
    'time', 'month', 'temperature', 'feelslike', 'wind',
    'direction', 'gust', 'cloud', 'humidity', 'precipitation',
    'pressure',
    'time_sin', 'time_cos',  # Biểu diễn thời gian trong ngày
    'month_sin', 'month_cos'  # Biểu diễn tháng trong năm
]

output_cols = [
    'temperature_after_3_hour',
    'feelslike_after_3_hour',
    'wind_after_3_hour',
    'direction_after_3_hour',
    'gust_after_3_hour',
    'cloud_after_3_hour',
    'humidity_after_3_hour',
    'precipitation_after_3_hour',
    'pressure_after_3_hour'
]


In [0]:
df = cast_columns_to_float(df, input_cols + output_cols)

In [0]:
df.printSchema()


root
 |-- time: float (nullable = true)
 |-- month: float (nullable = true)
 |-- year: string (nullable = true)
 |-- temperature: float (nullable = true)
 |-- feelslike: float (nullable = true)
 |-- wind: float (nullable = true)
 |-- gust: float (nullable = true)
 |-- cloud: float (nullable = true)
 |-- humidity: float (nullable = true)
 |-- precipitation: float (nullable = true)
 |-- pressure: float (nullable = true)
 |-- Rain: integer (nullable = true)
 |-- temperature_after_3_hour: float (nullable = true)
 |-- feelslike_after_3_hour: float (nullable = true)
 |-- wind_after_3_hour: float (nullable = true)
 |-- gust_after_3_hour: float (nullable = true)
 |-- cloud_after_3_hour: float (nullable = true)
 |-- humidity_after_3_hour: float (nullable = true)
 |-- precipitation_after_3_hour: float (nullable = true)
 |-- pressure_after_3_hour: float (nullable = true)
 |-- label: string (nullable = true)
 |-- rain_after_3_hour: integer (nullable = true)
 |-- direction: float (nullable = false)

In [0]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=input_cols, outputCol="features")
df = assembler.transform(df)


In [0]:
# Thư viện cần thiết
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, col

# Lặp qua từng cột đầu ra (output cần dự đoán) – ngoại trừ 'weather_after_3_hour' vì đây là phân loại
for target in output_cols:  # Bỏ weather_after_3_hour (classification)
    # Loại bỏ các dòng chứa giá trị null trong cột đặc trưng (input) và nhãn mục tiêu (target)
    current_df = df.dropna(subset=input_cols + [target])

    # Nếu sau khi lọc không còn dòng nào → bỏ qua mô hình này
    if current_df.count() == 0:
        print(f"⚠️ Bỏ qua {target} vì không còn dòng sau khi dropna.")
        continue

    print(f"🎯 Đang huấn luyện mô hình cho: {target}")

    # Tạo cột số thứ tự dòng để chia dữ liệu theo thời gian (giả lập thời gian bằng thứ tự)
    window = Window.orderBy(monotonically_increasing_id())
    indexed_df = current_df.withColumn("row_num", row_number().over(window))

    # Chia tập train/test theo thứ tự dòng (80% huấn luyện, 20% kiểm thử)
    total_rows = indexed_df.count()
    split_point = int(total_rows * 0.8)

    train_data = indexed_df.filter(col("row_num") <= split_point).drop("row_num")
    test_data = indexed_df.filter(col("row_num") > split_point).drop("row_num")

    # Khởi tạo và huấn luyện mô hình Random Forest hồi quy
    model = RandomForestRegressor(
        featuresCol="features",  # Cột đặc trưng đầu vào
        labelCol=target,         # Cột đầu ra cần dự đoán
        numTrees=100             # Số cây trong rừng
    )
    model_fitted = model.fit(train_data)

    # Dự đoán trên tập kiểm thử và đánh giá bằng chỉ số RMSE
    predictions = model_fitted.transform(test_data)
    evaluator = RegressionEvaluator(
        labelCol=target,
        predictionCol="prediction",
        metricName="rmse"       # Root Mean Squared Error
    )
    rmse = evaluator.evaluate(predictions)

    print(f"✅ RMSE cho {target}: {rmse:.3f}")

    # Lưu mô hình đã huấn luyện vào thư mục trên DBFS (Databricks File System)
    model_fitted.write().overwrite().save(f"dbfs:/minhhieu/delta/models/{target}_model")


🎯 Đang huấn luyện mô hình cho: temperature_after_3_hour
✅ RMSE cho temperature_after_3_hour: 1.877
🎯 Đang huấn luyện mô hình cho: feelslike_after_3_hour
✅ RMSE cho feelslike_after_3_hour: 1.613
🎯 Đang huấn luyện mô hình cho: wind_after_3_hour
✅ RMSE cho wind_after_3_hour: 2.428
🎯 Đang huấn luyện mô hình cho: direction_after_3_hour
✅ RMSE cho direction_after_3_hour: 3.391
🎯 Đang huấn luyện mô hình cho: gust_after_3_hour
✅ RMSE cho gust_after_3_hour: 4.306
🎯 Đang huấn luyện mô hình cho: cloud_after_3_hour
✅ RMSE cho cloud_after_3_hour: 22.324
🎯 Đang huấn luyện mô hình cho: humidity_after_3_hour
✅ RMSE cho humidity_after_3_hour: 4.184
🎯 Đang huấn luyện mô hình cho: precipitation_after_3_hour
✅ RMSE cho precipitation_after_3_hour: 1.972
🎯 Đang huấn luyện mô hình cho: pressure_after_3_hour
✅ RMSE cho pressure_after_3_hour: 0.904


In [0]:
# Thư viện cần thiết từ PySpark ML
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import col

# Chia dữ liệu thành tập huấn luyện và kiểm thử (80% train, 20% test)
train_data, test_data = df.randomSplit([0.8, 0.2], seed=42)

# Khởi tạo mô hình Random Forest Classifier
clf = RandomForestClassifier(
    featuresCol="features",                    # Cột chứa vector đặc trưng
    labelCol="weather_after_3_hour",           # Cột nhãn phân loại đầu ra
    numTrees=200,                              # Số cây trong rừng
    maxDepth=10                                # Độ sâu tối đa của mỗi cây
)

# Huấn luyện mô hình với tập train
model = clf.fit(train_data)

# Dự đoán trên tập test
predictions = model.transform(test_data)

# Khởi tạo bộ đánh giá Accuracy và F1-score
acc_evaluator = MulticlassClassificationEvaluator(
    labelCol="weather_after_3_hour",
    predictionCol="prediction",
    metricName="accuracy"                     # Tính độ chính xác
)

f1_evaluator = MulticlassClassificationEvaluator(
    labelCol="weather_after_3_hour",
    predictionCol="prediction",
    metricName="f1"                           # Tính chỉ số F1 (harmonic mean giữa precision & recall)
)

# Tính toán độ chính xác và F1-score trên tập kiểm thử
accuracy = acc_evaluator.evaluate(predictions)
f1_score = f1_evaluator.evaluate(predictions)

# In kết quả ra màn hình
print(f"✅ Accuracy for weather_after_3_hour: {accuracy:.3f}")
print(f"✅ F1-score for weather_after_3_hour: {f1_score:.3f}")

# Lưu mô hình đã huấn luyện vào hệ thống tệp (DBFS – Databricks File System)
model.write().overwrite().save("dbfs:/minhhieu/delta/models/weather_classifier_model")


✅ Accuracy for weather_after_3_hour: 0.583
✅ F1-score for weather_after_3_hour: 0.535


In [0]:
df.select("weather_after_3_hour").distinct().show()


+--------------------+
|weather_after_3_hour|
+--------------------+
|                 1.0|
|                 2.0|
|                 0.0|
|                 5.0|
|                 3.0|
|                 6.0|
|                 7.0|
|                 4.0|
|                12.0|
|                 9.0|
|                14.0|
|                10.0|
|                 8.0|
|                13.0|
|                11.0|
|                17.0|
|                16.0|
|                15.0|
|                19.0|
|                18.0|
+--------------------+



In [0]:
from pyspark.sql.functions import when

# Gán nhãn: các loại thời tiết liên quan đến mưa → 1, còn lại → 0
rain_labels = [3, 4, 6, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19]  # các giá trị mapping tương ứng với mưa

df = df.withColumn("label_after_3_hour", when(col("weather_after_3_hour").isin(rain_labels), 1).otherwise(0))

In [0]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import col

# Train-test split
train_data, test_data = df.randomSplit([0.8, 0.2], seed=42)

# RandomForestClassifier
clf = RandomForestClassifier(featuresCol="features", labelCol="label_after_3_hour", numTrees=200, maxDepth=10)

model = clf.fit(train_data)

# Dự đoán
predictions = model.transform(test_data)

# Đánh giá
acc_evaluator = MulticlassClassificationEvaluator(labelCol="label_after_3_hour", predictionCol="prediction", metricName="accuracy")
f1_evaluator = MulticlassClassificationEvaluator(labelCol="label_after_3_hour", predictionCol="prediction", metricName="f1")

accuracy = acc_evaluator.evaluate(predictions)
f1_score = f1_evaluator.evaluate(predictions)

print(f"✅ Accuracy for weather_after_3_hour: {accuracy:.3f}")
print(f"✅ F1-score for weather_after_3_hour: {f1_score:.3f}")

model.write().overwrite().save("dbfs:/minhhieu/delta/models/weather_classifier_model_2")


✅ Accuracy for weather_after_3_hour: 0.876
✅ F1-score for weather_after_3_hour: 0.870
